In [ ]:
# Sharpe Ratio of S&P500 Futures 





$$SR = (r_p -r_f)/\sigma_p$$

In [ ]:
!pip install yfinance --upgrade --no-cache-dir

In [1]:
import yfinance as yf 
import numpy as np 

In [ ]:
data = yf.download("ES=F")
prices = data["Close"]
prices

[*********************100%***********************]  1 of 1 completed


Ticker,ES=F
Date,
2000-09-18,1467.50
2000-09-19,1478.50
2000-09-20,1469.50
2000-09-21,1469.50
2000-09-22,1468.50
...,...
2025-02-18,6146.75
2025-02-19,6163.00
2025-02-20,6136.50


In [7]:
returns = prices.pct_change()
returns 

Ticker,ES=F
Date,
2000-09-18,NaN
2000-09-19,0.007496
2000-09-20,-0.006087
2000-09-21,0.000000
2000-09-22,-0.000681
...,...
2025-02-18,0.000122
2025-02-19,0.002644
2025-02-20,-0.004300


In [14]:
returns= returns.dropna()
returns 

Ticker,ES=F
Date,
2000-09-19,0.007496
2000-09-20,-0.006087
2000-09-21,0.000000
2000-09-22,-0.000681
2000-09-25,-0.005107
...,...
2025-02-18,0.000122
2025-02-19,0.002644
2025-02-20,-0.004300


In [15]:
#number of days per year
periods_per_year = 252
# number of days
n_periods = prices.shape[0]
n_periods

6171

In [16]:
annualized_returns = (1+ prices).prod()**(periods_per_year/n_periods) -1 
annualized_returns

/home/codespace/.local/lib/python3.12/site-packages/numpy/_core/_methods.py:56: RuntimeWarning: overflow encountered in reduce
  return umr_prod(a, axis, dtype, out, keepdims, initial, where)


Ticker
ES=F    inf
dtype: float64

# We encounter Overflow Error 
/home/codespace/.local/lib/python3.12/site-packages/numpy/_core/_methods.py:56: RuntimeWarning: overflow encountered in reduce
  return umr_prod(a, axis, dtype, out, keepdims, initial, where)

In [ ]:
# STABLE VERSION: Use log returns to avoid overflow
log_returns = np.log1p(returns)  # Equivalent to ln(1 + r_t)
cumulative_log_return = log_returns.sum()

# Convert back to arithmetic return
cumulative_return = np.exp(cumulative_log_return) - 1

# Annualize (252 trading days/year)
total_days = len(returns)
annualized_return = (1 + cumulative_return) ** (252 / total_days) - 1

print(f"Annualized Return: {annualized_return:.2%}")

# Compute Stable Version of the Compounded Return 


In [49]:
#natural log is np.log
log_returns = np.log(1+  returns )
compounded_log_return = log_returns.sum()
# convert to returns 
compounded_return = np.exp(compounded_log_return)  -1 
# annualize (252 trading days/year)
#number of days per year
periods_per_year = 252
# number of days
n_periods = prices.shape[0]

#format to 0+return instead of 1+return
annualized_return = (1+ compounded_return )**(periods_per_year/n_periods) -1 
annualized_return = annualized_return.values[0]
print(f"Annualized Return:  {annualized_return*100: .2f}%")




Annualized Return:   5.95%


In [36]:
annualized_volatility = (returns.std()*(252)**(1/2)).values[0]
annualized_volatility

np.float64(0.19404296651229747)

## Arithmethic Returns (Not suitable for evaluating several periods)

In [33]:
annualized_returns_arith = (returns.mean()*252).values[0]
print(f"Annualized Arithmetic Return:  {annualized_returns_arith*100: .2f}%")


Annualized Arithmetic Return:   7.67%


## Getting the Risk Free Rate

In [37]:
sp500_start_date = prices.index[0]
sp500_start_date

Timestamp('2000-09-18 00:00:00')

In [38]:
sp500_end_date = prices.index[-1]
sp500_end_date

Timestamp('2025-02-23 00:00:00')

In [50]:
risk_free_rate = yf.download("^IRX",
                             start = sp500_start_date )["Close"]
risk_free_rate

[*********************100%***********************]  1 of 1 completed


Ticker,^IRX
Date,
2000-09-18,5.950
2000-09-19,5.970
2000-09-20,5.970
2000-09-21,5.960
2000-09-22,5.970
...,...
2025-02-14,4.213
2025-02-18,4.213
2025-02-19,4.218


In [45]:
avg_risk_free_rate = risk_free_rate.mean().values[0]
# express in decimals 
avg_risk_free_rate /= 100
avg_risk_free_rate

np.float64(0.016844398813558432)

## Sharpe Ratio

In [51]:
sharpe_ratio = (annualized_return-avg_risk_free_rate) /annualized_volatility
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")

Sharpe Ratio: 0.2200
